Inference notebook to determine the position and angle of RF connector head
using the circuit_rfHeadRotation image data set
- model used random augmentation of position and rotation

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import math

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

In [ ]:
import sys
print(sys.modules['keras_retinanet'].__file__)
print(sys.modules['keras_resnet'].__file__)
print(sys.modules['keras'].__file__)

In [ ]:
model = keras.models.load_model('../snapshots/circuit_rfHeadRotation/circuit_resnet50_csv_10.h5', custom_objects=custom_objects)
# print(model.summary())

In [ ]:
model_json = model.to_json()
print(model_json)

In [ ]:
def add_noise(inData):
    result = np.random.normal(inData, scale=0.5).astype(np.uint8)
    print("adding noise ---- from {}:{} to {}:{}".format(inData.min(), inData.max(), result.min(), result.max()))
    return result

def set_to_zero(inData):
    result = np.zeros(inData.shape).astype(np.uint8)
    print("adding noise ---- from {}:{} to {}:{}".format(inData.min(), inData.max(), result.min(), result.max()))
    return result



In [ ]:

def get2DPose(inFile):
    image = cv2.imread(inFile)
    print(image.shape)
    
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = validation_generator.preprocess_image(image)
    print(image.shape)
    image, scale = validation_generator.resize_image(image)
    print(image.shape)
    print(scale)
    
    # process image
    start = time.time()
    detections = None
    _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]
    # correct for image scale
    detections[0, :, :4] /= scale
    
    b = None

    # visualize detections
    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
        if score < 0.5:
            continue
            
        b = detections[0, idx, :4].astype(int)
        color = (0, 0, 255) if label == 0 else (255,0,0)
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), color, 3)
        angleResult = validation_generator.label_to_name(label)
        caption = "{} {:.2f}".format(angleResult, score)
        cv2.putText(draw, caption, (b[0], b[1] - 2), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 3)
        cv2.putText(draw, caption, (b[0], b[1] - 2), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)

        centerX = (b[2]+b[0])/2
        centerY = (b[3]+b[1])/2
        print(angleResult, centerX, centerY, score)
        
    
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()

    


# Tests

In [ ]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

validation_generator = CSVGenerator(
                '/media/kai/Data1/robotPrototype/data/180419rfHeadRotation/annotationRFHead.txt',
                '/media/kai/Data1/robotPrototype/data/180419rfHeadRotation/classnameRFHead.txt',
                val_image_data_generator,
                batch_size=10
            )

In [ ]:
import random
#inAngle = random.randint(0,8)*45
#testFile = "/media/kai/Data1/robotPrototype/data/180419rfHeadRotation/{}/5-2.bmp".format(inAngle)
#print("inAngle: ", inAngle)


index = random.randint(0,validation_generator.size()-1)
testFile = validation_generator.image_path(index)
print("imagePath: ", testFile)
get2DPose(testFile)
